In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)

RAW = Path("../data/raw").resolve()
print("RAW exists:", RAW.exists())
print("RAW path:", RAW)


RAW exists: True
RAW path: /home/danyak/olist-ecommerce-analytics/data/raw


In [2]:
files = sorted([p.name for p in RAW.glob("*.csv")])
print("CSV files count:", len(files))
files


CSV files count: 9


['olist_customers_dataset.csv',
 'olist_geolocation_dataset.csv',
 'olist_order_items_dataset.csv',
 'olist_order_payments_dataset.csv',
 'olist_order_reviews_dataset.csv',
 'olist_orders_dataset.csv',
 'olist_products_dataset.csv',
 'olist_sellers_dataset.csv',
 'product_category_name_translation.csv']

In [3]:
customers = pd.read_csv(RAW / "olist_customers_dataset.csv")

orders = pd.read_csv(
    RAW / "olist_orders_dataset.csv",
    parse_dates=[
        "order_purchase_timestamp",
        "order_approved_at",
        "order_delivered_carrier_date",
        "order_delivered_customer_date",
        "order_estimated_delivery_date",
    ],
)

items = pd.read_csv(RAW / "olist_order_items_dataset.csv")
payments = pd.read_csv(RAW / "olist_order_payments_dataset.csv")

reviews = pd.read_csv(
    RAW / "olist_order_reviews_dataset.csv",
    parse_dates=["review_creation_date", "review_answer_timestamp"],
)

products = pd.read_csv(RAW / "olist_products_dataset.csv")
cat_tr = pd.read_csv(RAW / "product_category_name_translation.csv")


In [4]:
for name, df in {
    "customers": customers, "orders": orders, "items": items,
    "payments": payments, "reviews": reviews, "products": products
}.items():
    print(f"{name:10s} {df.shape}")


customers  (99441, 5)
orders     (99441, 8)
items      (112650, 7)
payments   (103886, 5)
reviews    (99224, 7)
products   (32951, 9)


In [5]:
print("orders: order_id unique? ->", orders["order_id"].is_unique)
print("orders: unique order_id ->", orders["order_id"].nunique(), "rows ->", len(orders))

assert orders["order_id"].is_unique, "order_id в orders должен быть уникален"


orders: order_id unique? -> True
orders: unique order_id -> 99441 rows -> 99441


In [6]:
print("items cover orders:", items["order_id"].nunique(), "/", orders["order_id"].nunique())
print("payments cover orders:", payments["order_id"].nunique(), "/", orders["order_id"].nunique())
print("reviews cover orders:", reviews["order_id"].nunique(), "/", orders["order_id"].nunique())


items cover orders: 98666 / 99441
payments cover orders: 99440 / 99441
reviews cover orders: 98673 / 99441


In [7]:
items_agg = (
    items.groupby("order_id", as_index=False)
    .agg(
        items_cnt=("order_item_id", "count"),      # сколько позиций в заказе
        sellers_cnt=("seller_id", "nunique"),      # сколько уникальных продавцов
        products_cnt=("product_id", "nunique"),    # сколько уникальных товаров
        gmv_items=("price", "sum"),                # сумма цен товаров
        freight_total=("freight_value", "sum"),    # сумма доставки
    )
)

items_agg["gmv_total"] = items_agg["gmv_items"] + items_agg["freight_total"]
items_agg.head()


,order_id,items_cnt,sellers_cnt,products_cnt,gmv_items,freight_total,gmv_total
0,00010242fe8c5a6d1ba2dd792cb16214,1,1,1,58.90,13.29,72.19
1,00018f77f2f0320c557190d7a144bdd3,1,1,1,239.90,19.93,259.83
2,000229ec398224ef6ca0657da4fc703e,1,1,1,199.00,17.87,216.87
3,00024acbcdf0a6daa1e931b038114c75,1,1,1,12.99,12.79,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,1,1,199.90,18.14,218.04


In [8]:
print("items_agg rows:", len(items_agg), "unique order_id:", items_agg["order_id"].nunique())
items_agg[["gmv_items", "freight_total", "gmv_total", "items_cnt"]].describe()

print("Negative gmv_items share:", (items_agg["gmv_items"] < 0).mean())
print("Negative freight_total share:", (items_agg["freight_total"] < 0).mean())


items_agg rows: 98666 unique order_id: 98666
Negative gmv_items share: 0.0
Negative freight_total share: 0.0


In [9]:
payments_agg = (
    payments.groupby("order_id", as_index=False)
    .agg(
        payment_value=("payment_value", "sum"),            # сумма всех платежей по заказу
        payments_cnt=("payment_sequential", "count"),      # сколько строк платежей
        installments_max=("payment_installments", "max"),  # максимальная рассрочка
    )
)

payment_type_main = (
    payments.groupby("order_id")["payment_type"]
    .agg(lambda s: s.value_counts().index[0])             # самый частый тип оплаты в заказе
    .reset_index(name="payment_type_main")
)

payments_agg.head(), payment_type_main.head()


(                           order_id  payment_value  payments_cnt  installments_max
 0  00010242fe8c5a6d1ba2dd792cb16214          72.19             1                 2
 1  00018f77f2f0320c557190d7a144bdd3         259.83             1                 3
 2  000229ec398224ef6ca0657da4fc703e         216.87             1                 5
 3  00024acbcdf0a6daa1e931b038114c75          25.78             1                 2
 4  00042b26cf59d7ce69dfabb4e55b4fd9         218.04             1                 3,
                            order_id payment_type_main
 0  00010242fe8c5a6d1ba2dd792cb16214       credit_card
 1  00018f77f2f0320c557190d7a144bdd3       credit_card
 2  000229ec398224ef6ca0657da4fc703e       credit_card
 3  00024acbcdf0a6daa1e931b038114c75       credit_card
 4  00042b26cf59d7ce69dfabb4e55b4fd9       credit_card)

In [10]:
print("payments_agg rows:", len(payments_agg), "unique order_id:", payments_agg["order_id"].nunique())
print("Negative payment_value share:", (payments_agg["payment_value"] < 0).mean())

payment_type_main["payment_type_main"].value_counts()


payments_agg rows: 99440 unique order_id: 99440
Negative payment_value share: 0.0


payment_type_main
credit_card    75270
boleto         19784
voucher         2856
debit_card      1527
not_defined        3
Name: count, dtype: int64

In [11]:
reviews_agg = (
    reviews.groupby("order_id", as_index=False)
    .agg(
        review_score=("review_score", "mean"),        # средняя оценка по заказу
        review_cnt=("review_id", "nunique"),         # сколько уникальных отзывов
        review_created=("review_creation_date", "min")# когда появился отзыв
    )
)

reviews_agg.head()


,order_id,review_score,review_cnt,review_created
0,00010242fe8c5a6d1ba2dd792cb16214,5.0,1,2017-09-21
1,00018f77f2f0320c557190d7a144bdd3,4.0,1,2017-05-13
2,000229ec398224ef6ca0657da4fc703e,5.0,1,2018-01-23
3,00024acbcdf0a6daa1e931b038114c75,4.0,1,2018-08-15
4,00042b26cf59d7ce69dfabb4e55b4fd9,5.0,1,2017-03-02


In [12]:
print("reviews_agg rows:", len(reviews_agg), "unique order_id:", reviews_agg["order_id"].nunique())

reviews_agg["review_score"].value_counts(dropna=False).sort_index()

bad_scores_share = (~reviews_agg["review_score"].between(1, 5)).mean()
print("Bad review_score share:", bad_scores_share)


reviews_agg rows: 98673 unique order_id: 98673
Bad review_score share: 0.0


In [13]:
products_cat = (
    products.merge(cat_tr, on="product_category_name", how="left")
    [["product_id", "product_category_name_english"]]
)
products_cat.head()


,product_id,product_category_name_english
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,art
2,96bd76ec8810374ed1b65e291975717f,sports_leisure
3,cef67bcfe19066a932b7673e239eb23d,baby
4,9dc1a7de274444849c219cff195d0b71,housewares


In [14]:
items_with_cat = items.merge(products_cat, on="product_id", how="left")
items_with_cat[["order_id", "product_id", "product_category_name_english", "price"]].head()


,order_id,product_id,product_category_name_english,price
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,cool_stuff,58.90
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,pet_shop,239.90
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,furniture_decor,199.00
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,perfumery,12.99
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,garden_tools,199.90


In [15]:
order_category = (
    items_with_cat
    .groupby(["order_id", "product_category_name_english"], as_index=False)
    .agg(category_gmv=("price", "sum"))
    .sort_values(["order_id", "category_gmv"], ascending=[True, False])
)

order_category_main = (
    order_category.drop_duplicates("order_id")[["order_id", "product_category_name_english"]]
    .rename(columns={"product_category_name_english": "category_main"})
)

order_category_main.head()


,order_id,category_main
0,00010242fe8c5a6d1ba2dd792cb16214,cool_stuff
1,00018f77f2f0320c557190d7a144bdd3,pet_shop
2,000229ec398224ef6ca0657da4fc703e,furniture_decor
3,00024acbcdf0a6daa1e931b038114c75,perfumery
4,00042b26cf59d7ce69dfabb4e55b4fd9,garden_tools


In [16]:
print("order_category_main rows:", len(order_category_main))
print("share of missing category_main:", order_category_main["category_main"].isna().mean())

order_category_main["category_main"].value_counts(dropna=False).head(10)


order_category_main rows: 97256
share of missing category_main: 0.0


category_main
bed_bath_table           9351
health_beauty            8802
sports_leisure           7684
computers_accessories    6671
furniture_decor          6333
housewares               5828
watches_gifts            5606
telephony                4179
auto                     3880
toys                     3872
Name: count, dtype: int64

In [17]:
mart = (
    orders
    .merge(customers, on="customer_id", how="left")
    .merge(items_agg, on="order_id", how="left")
    .merge(payments_agg, on="order_id", how="left")
    .merge(payment_type_main, on="order_id", how="left")
    .merge(reviews_agg, on="order_id", how="left")
    .merge(order_category_main, on="order_id", how="left")
)

print("mart rows:", len(mart))
print("mart unique orders:", mart["order_id"].nunique())


mart rows: 99441
mart unique orders: 99441


In [18]:
assert len(mart) == len(orders), "После merge число строк изменилось — проверь ключи и агрегаты"
assert mart["order_id"].is_unique, "В витрине order_id должен быть уникален"


In [19]:
mart[["gmv_total", "payment_value", "review_score", "category_main"]].isna().mean().sort_values(ascending=False)


category_main    0.021973
gmv_total        0.007794
review_score     0.007723
payment_value    0.000010
dtype: float64

In [20]:
mart["delivered"] = mart["order_delivered_customer_date"].notna()

mart["delivery_days"] = (
    mart["order_delivered_customer_date"] - mart["order_purchase_timestamp"]
).dt.total_seconds() / 86400

mart["delay_days"] = (
    mart["order_delivered_customer_date"] - mart["order_estimated_delivery_date"]
).dt.total_seconds() / 86400

mart["on_time"] = mart["delay_days"].le(0)

mart["purchase_month"] = mart["order_purchase_timestamp"].dt.to_period("M").astype(str)
mart["bad_review"] = mart["review_score"].le(2)


In [21]:
print("Delivered rate:", round(mart["delivered"].mean(), 3))
print("Avg review:", round(mart["review_score"].mean(), 3))

mart[["delivery_days", "delay_days"]].describe()


Delivered rate: 0.97
Avg review: 4.087


,delivery_days,delay_days
count,96476.000000,96476.000000
mean,12.558702,-11.179120
std,9.546530,10.186113
min,0.533414,-146.016123
25%,6.766403,-16.244384
50%,10.217755,-11.948941
75%,15.720327,-6.390000
max,209.628611,188.975081


In [22]:
kpi_month = (
    mart.groupby("purchase_month", as_index=False)
    .agg(
        orders=("order_id", "nunique"),
        customers=("customer_unique_id", "nunique"),
        gmv=("gmv_total", lambda s: s.sum(min_count=1)),
        aov=("gmv_total", "mean"),
        delivered_rate=("delivered", "mean"),
        on_time_rate=("on_time", "mean"),
        avg_delivery_days=("delivery_days", "mean"),
        avg_delay_days=("delay_days", "mean"),
        avg_review=("review_score", "mean"),
        bad_review_rate=("bad_review", "mean"),
    )
    .sort_values("purchase_month")
)

kpi_month.head()


,purchase_month,orders,customers,gmv,aov,delivered_rate,on_time_rate,avg_delivery_days,avg_delay_days,avg_review,bad_review_rate
0,2016-09,4,4,354.75,118.250000,0.250000,0.000000,54.813194,36.324745,1.000000,1.000000
1,2016-10,324,321,56808.84,184.444286,0.833333,0.824074,19.578572,-36.059479,3.561129,0.287037
2,2016-12,1,1,19.62,19.620000,1.000000,1.000000,4.693021,-21.336991,5.000000,0.000000
3,2017-01,800,765,137188.49,173.876413,0.937500,0.908750,12.647044,-26.861788,4.062658,0.153750
4,2017-02,1780,1755,286280.62,165.193664,0.928652,0.898876,13.168825,-18.680104,4.015837,0.160674


In [23]:
kpi_month.tail()


,purchase_month,orders,customers,gmv,aov,delivered_rate,on_time_rate,avg_delivery_days,avg_delay_days,avg_review,bad_review_rate
20,2018-06,6167,6128,1022677.11,166.019011,0.988487,0.975028,9.239216,-18.533524,4.277479,0.107346
21,2018-07,6292,6230,1058728.03,168.775391,0.978385,0.934520,8.958636,-10.731490,4.263246,0.110140
22,2018-08,6512,6460,1003308.47,155.503483,0.975276,0.873925,7.732780,-7.453550,4.256207,0.108876
23,2018-09,16,14,166.46,166.460000,0.000000,0.000000,NaN,NaN,1.800000,0.750000
24,2018-10,4,4,NaN,NaN,0.000000,0.000000,NaN,NaN,2.250000,0.750000


In [24]:
tmp = mart[mart["review_score"].notna()].copy()

# Создаем категорию для не доставленных заказов
tmp["delay_bucket"] = np.where(
    tmp["delivered"],
    pd.cut(
        tmp["delay_days"],
        bins=[-999, -10, -5, -2, 0, 2, 5, 10, 999],
        labels=["<=-10", "-10..-5", "-5..-2", "-2..0", "0..2", "2..5", "5..10", "10+"]
    ),
    "Not delivered"
)

delay_vs_review = (
    tmp.groupby(["delivered", "delay_bucket"], as_index=False, observed=True)
    .agg(
        orders=("order_id", "nunique"),
        avg_review=("review_score", "mean"),
        bad_review_rate=("bad_review", "mean"),
        avg_delay=("delay_days", "mean"),
    )
)

delay_vs_review


,delivered,delay_bucket,orders,avg_review,bad_review_rate,avg_delay
0,False,Not delivered,2843,1.753254,0.776996,NaN
1,True,-10..-5,22442,4.278184,0.091347,-7.375671
2,True,-2..0,2990,4.140970,0.113378,-0.813746
3,True,-5..-2,5826,4.157398,0.108823,-3.403831
4,True,0..2,2100,3.916905,0.151905,1.076815
5,True,10+,2233,1.699507,0.790864,22.846366
6,True,2..5,1468,2.809264,0.461853,3.615746
7,True,5..10,1861,1.891725,0.737775,7.411616
8,True,<=-10,56910,4.322495,0.089281,-16.858884


In [25]:
tmp[tmp["delivered"]== False]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,items_cnt,sellers_cnt,products_cnt,gmv_items,freight_total,gmv_total,payment_value,payments_cnt,installments_max,payment_type_main,review_score,review_cnt,review_created,category_main,delivered,delivery_days,delay_days,on_time,purchase_month,bad_review,delay_bucket
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11 12:22:08,2017-04-13 13:25:17,NaT,NaT,2017-05-09,36edbb3fb164b1f16485364b6fb04c73,98900,santa rosa,RS,1.0,1.0,1.0,49.90,16.05,65.95,65.95,1.0,1.0,credit_card,2.0,1.0,2017-05-13,NaN,False,NaN,NaN,False,2017-04,True,Not delivered
44,ee64d42b8cf066f35eac1cf57de1aa85,caded193e8e47b8362864762a83db3c5,shipped,2018-06-04 16:44:48,2018-06-05 04:31:18,2018-06-05 14:32:00,NaT,2018-06-28,08fb46d35bb3ab4037202c23592d1259,13215,jundiai,SP,1.0,1.0,1.0,14.49,7.87,22.36,22.36,1.0,1.0,boleto,1.0,1.0,2018-07-01,health_beauty,False,NaN,NaN,False,2018-06,True,Not delivered
103,0760a852e4e9d89eb77bf631eaaf1c84,d2a79636084590b7465af8ab374a8cf5,invoiced,2018-08-03 17:44:42,2018-08-07 06:15:14,NaT,NaT,2018-08-21,c7f8d7b1fffc946d7069574f74c39f4e,88140,santo amaro da imperatriz,SC,1.0,1.0,1.0,35.00,15.35,50.35,50.35,1.0,1.0,boleto,3.0,1.0,2018-08-25,christmas_supplies,False,NaN,NaN,False,2018-08,False,Not delivered
128,15bed8e2fec7fdbadb186b57c46c92f2,f3f0e613e0bdb9c7cee75504f0f90679,processing,2017-09-03 14:22:03,2017-09-03 14:30:09,NaT,NaT,2017-10-03,9f269af9c49244f6ba4a46985a3cfc2e,3436,sao paulo,SP,1.0,1.0,1.0,125.90,12.38,138.28,138.28,1.0,2.0,credit_card,5.0,1.0,2017-10-05,perfumery,False,NaN,NaN,False,2017-09,False,Not delivered
162,36530871a5e80138db53bcfd8a104d90,4dafe3c841d2d6cc8a8b6d25b35704b9,shipped,2017-05-09 11:48:37,2017-05-11 11:45:14,2017-05-11 13:21:47,NaT,2017-06-08,10669e874b7ddaab97d6d539578576d7,54762,camaragibe,PE,1.0,1.0,1.0,36.90,26.89,63.79,63.79,1.0,6.0,credit_card,1.0,1.0,2017-06-14,sports_leisure,False,NaN,NaN,False,2017-05,True,Not delivered
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99283,3a3cddda5a7c27851bd96c3313412840,0b0d6095c5555fe083844281f6b093bb,canceled,2018-08-31 16:13:44,NaT,NaT,NaT,2018-10-01,e90598185d2427a35e32ef241a5c04aa,11075,santos,SP,NaN,NaN,NaN,NaN,NaN,NaN,63.89,1.0,1.0,voucher,5.0,1.0,2018-07-24,NaN,False,NaN,NaN,False,2018-08,False,Not delivered
99313,e9e64a17afa9653aacf2616d94c005b8,b4cd0522e632e481f8eaf766a2646e86,processing,2018-01-05 23:07:24,2018-01-09 07:18:05,NaT,NaT,2018-02-06,965191786b70912f793e4a777fd623cd,13050,campinas,SP,1.0,1.0,1.0,129.90,14.66,144.56,144.56,1.0,1.0,boleto,1.0,1.0,2018-02-10,watches_gifts,False,NaN,NaN,False,2018-01,True,Not delivered
99347,a89abace0dcc01eeb267a9660b5ac126,2f0524a7b1b3845a1a57fcf3910c4333,canceled,2018-09-06 18:45:47,NaT,NaT,NaT,2018-09-27,d05c44a138277ad325d915c6b7ccbcdf,5344,sao paulo,SP,NaN,NaN,NaN,NaN,NaN,NaN,107.13,1.0,1.0,voucher,4.0,1.0,2018-07-31,NaN,False,NaN,NaN,False,2018-09,False,Not delivered
99348,a69ba794cc7deb415c3e15a0a3877e69,726f0894b5becdf952ea537d5266e543,unavailable,2017-08-23 16:28:04,2017-08-28 15:44:47,NaT,NaT,2017-09-15,e72a90a2b29fe1a8795b284aaaa3246f,22723,rio de janeiro,RJ,NaN,NaN,NaN,NaN,NaN,NaN,195.91,1.0,2.0,credit_card,1.0,1.0,2017-09-20,NaN,False,NaN,NaN,False,2017-08,True,Not delivered


In [26]:
tmp2 = mart.copy()
tmp2["on_time_delivered_only"] = np.where(tmp2["delivered"], tmp2["on_time"], np.nan)

check = (
    tmp2.groupby("purchase_month", as_index=False)
    .agg(
        delivered_rate=("delivered", "mean"),
        on_time_rate_delivered=("on_time_delivered_only", "mean"),
    )
    .sort_values("purchase_month")
)

check.head()


,purchase_month,delivered_rate,on_time_rate_delivered
0,2016-09,0.250000,0.000000
1,2016-10,0.833333,0.988889
2,2016-12,1.000000,1.000000
3,2017-01,0.937500,0.969333
4,2017-02,0.928652,0.967937


In [27]:
OUT = Path("../data/processed").resolve()
OUT.mkdir(parents=True, exist_ok=True)

mart.to_csv(OUT / "mart_orders.csv", index=False)
kpi_month.to_csv(OUT / "kpi_month.csv", index=False)
delay_vs_review.to_csv(OUT / "delay_vs_review.csv", index=False)

print("Saved files:")
for p in [OUT / "mart_orders.csv", OUT / "kpi_month.csv", OUT / "delay_vs_review.csv"]:
    print(" -", p.name, "MB:", round(p.stat().st_size / 1024 / 1024, 2))


Saved files:
 - mart_orders.csv MB: 35.83
 - kpi_month.csv MB: 0.0
 - delay_vs_review.csv MB: 0.0


In [28]:
print("Orders total:", mart["order_id"].nunique())
print("Delivered rate:", round(mart["delivered"].mean(), 3))
print("Avg review:", round(mart["review_score"].mean(), 3))
print("Median delay_days (delivered):", round(mart.loc[mart["delivered"], "delay_days"].median(), 2))
print("Share of bad reviews (<=2):", round(mart["bad_review"].mean(), 3))


Orders total: 99441
Delivered rate: 0.97
Avg review: 4.087
Median delay_days (delivered): -11.95
Share of bad reviews (<=2): 0.145
